In [1]:
import requests
import csv
from datetime import datetime, timedelta

In [2]:
MIN_BASE_FEE_PER_BLOB_GAS = 1
TARGET_BLOB_GAS_PER_BLOCK = 786432 # 6 blobs
BLOB_BASE_FEE_UPDATE_FRACTION_PRAGUE = 5007716

In [3]:
def fake_exponential(factor: int, numerator: int, denominator: int) -> int:
    _numerator = int(numerator)
    _denominator = int(denominator)
    i = int(1)
    output = int(0)
    numerator_accum = int(factor * denominator)
    while numerator_accum > 0:
        output += numerator_accum
        temp = numerator_accum * _numerator
        numerator_accum = (temp) // (_denominator * i)
        i += 1
    return output // _denominator

def get_base_fee_per_blob_gas(min_base_fee_per_blob_gas: int, excess_blob_gas: int, blob_base_fee_update_fraction: int) -> int:
    return fake_exponential(
        min_base_fee_per_blob_gas,
        excess_blob_gas,
        blob_base_fee_update_fraction
    )

In [ ]:


url = 'https://api.blobscan.com/blocks'
pectra_fork_blocknumber = 22431084 # the block number of the Pactra fork
start_block = pectra_fork_blocknumber
end_block = pectra_fork_blocknumber + (5 * 60 * 24 * 7)

data = []
selected_keys = ['number', 'timestamp', 'blobGasUsed', 'excessBlobGas', 'blobGasPrice']


block = start_block
while block < end_block:
    response = requests.get(
        url,
        params={
            'startBlock': block,
            'sort': 'asc',
            'count': 'true',
        }
    ).json()

    data.extend([
        {key: d[key] for key in selected_keys if key in d} for d in response['blocks']
    ])

    block = response['blocks'][-1]['number'] + 1


index = 1
while index < len(data):
    if (data[index]['number'] - data[index - 1]['number']) > 1:
        time = datetime.fromisoformat(data[index - 1]['timestamp']) + timedelta(seconds=12)
        excessBlobGas = max(0, int(data[index - 1]['excessBlobGas']) + int(data[index - 1]['blobGasUsed']) - TARGET_BLOB_GAS_PER_BLOCK)
        blobGasPrice = get_base_fee_per_blob_gas(MIN_BASE_FEE_PER_BLOB_GAS, excessBlobGas, BLOB_BASE_FEE_UPDATE_FRACTION_PRAGUE)
        data.insert(index, {
            'number': data[index - 1]['number'] + 1,
            'timestamp': time.isoformat(timespec='milliseconds').replace('+00:00', 'Z'),
            'blobGasUsed': 0,
            'excessBlobGas': excessBlobGas,
            'blobGasPrice': blobGasPrice,
        })
    index += 1


with open('data/blocks_1week_after_pectra.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=selected_keys)
    writer.writeheader()
    writer.writerows(data)